In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import geocoder

path = '/Users/216cjl/Local_Data/Datathon_Data/'
path_to_dataset = '/Users/216cjl/Local_Data/Datathon_Data/BigSupplyCo_Data_Files /'

customers = pd.read_csv(path_to_dataset + 'BigSupplyCo_Customers.csv')
orders = pd.read_csv(path_to_dataset + 'BigSupplyCo_Orders.csv')
products = pd.read_csv(path_to_dataset + 'BigSupplyCo_Products.csv')
departments = pd.read_csv(path_to_dataset + 'BigSupplyCo_Departments_Cleaned.csv')
categories = pd.read_csv(path_to_dataset + 'BigSupplyCo_Categories.csv')

customers['Customer Full Name'] = customers['Customer Fname'].astype(str)+ \
                                  ' ' + customers['Customer Lname'].astype(str)
customers['Customer Zipcode'] = customers['Customer Zipcode'].fillna(0)
customers = customers.drop(['Customer Email', 
                            'Customer Password',
                            'Customer Fname',
                            'Customer Lname',
                            'Customer Street'], axis=1)
                            #'Customer Zipcode'], axis=1) shall we drop this?
orders = orders.drop(['Order Zipcode',  # too few non-null values
                      'Order Item Discount',
#                       'Order Item Discount Rate',
                      'Sales',  # this and two above determine Order Item Total - after discounts
                      'Order Item Id',  # does not join with any other column in a different table
#                      'Order Item Quantity',  # I think the quantity is not important: perfection should occur for every order
                      'Unnamed: 0',
                      'Order Id'], axis=1) 
products = products.drop(['Product Description',  # NaN everywhere, hence remove it
                          'Product Image',  # useless, hence remove it 
                          'Product Status',  # only one value for this column, hence remove it
                          'Product Name'], axis=1)  # group by categories rather than by each of the same category

def get_country(row):
    return geocoder.osm([row['Latitude'], row['Longitude']], method='reverse').json['country']
# departments = departments.drop(['Latitude',
#                                 'Longitude'], axis=1)

join_dept = pd.merge(orders, departments, left_on='Order Department Id', right_on='Department Id').drop(
            columns=['Department Id'])
join_cust = pd.merge(join_dept, customers, left_on='Order Customer Id', right_on='Customer Id').drop(
            columns=['Customer Id'])
join_prod = pd.merge(join_cust, products, left_on='Order Item Cardprod Id', right_on='Product Card Id').drop(
            columns=['Product Card Id'])
data_ready = pd.merge(join_prod, categories, left_on='Product Category Id', right_on='Category Id').drop(
            columns=['Category Id'])

data_ready['Date'] = pd.to_datetime(data_ready['order date (DateOrders)'],
                                    infer_datetime_format=True).dt.date
data_ready.drop(columns=['order date (DateOrders)'], inplace=True)

data_ready['Date'] = pd.to_datetime(data_ready['Date'], infer_datetime_format=True).astype('datetime64[ns]')
data_ready.set_index('Date', inplace=True)
data_ready.sort_index(inplace=True)

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#data_ready = reduce_mem_usage(data_ready)

def handle_non_numerical_data(df):
    columns = df.columns.values
    print(columns)
    for column in columns:
        text_digit_vals = {}
#         def convert_to_int(val):
#             return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

#         df[column] = list(map(convert_to_int, df[column]))
    return df

df = data_ready.copy()
data_ready_only_numbers = handle_non_numerical_data(df)

print(data_ready.head(5))
print(data_ready_only_numbers.head(5))

['Order Item Cardprod Id' 'Order Customer Id' 'Order Department Id'
 'Market' 'Order City' 'Order Country' 'Order Region' 'Order State'
 'Order Status' 'Order Item Discount Rate' 'Order Item Quantity'
 'Order Item Total' 'Order Profit' 'Type' 'Days for shipping (real)'
 'Days for shipment (scheduled)' 'Delivery Status' 'Late Delivery Risk'
 'Department Name' 'Latitude' 'Longitude' 'Customer City'
 'Customer Country' 'Customer Segment' 'Customer State' 'Customer Zipcode'
 'Customer Full Name' 'Product Category Id' 'Product Price'
 'Category Name']
            Order Item Cardprod Id  Order Customer Id  Order Department Id  \
Date                                                                         
2015-01-01                     403               5225                    4   
2015-01-01                     134               1837                    2   
2015-01-01                     957               4189                    7   
2015-01-01                     365               1205    

In [3]:
data_ready = data_ready.rename(columns = {'Latitude': 'Depart_Latitude', 'Longitude': 'Depart_Longitude'})
# data_ready = data_ready[~(data_ready['Customer Full Name'] == 'Mary Smith')]

## Distance

In [4]:
citys=pd.read_table(path+'cities5000.txt', header = None)
citys = citys.rename(columns = {1:'City' ,4: 'Latitude', 5:'Longtitude', 8:'ISO'})
citys = citys[['City','Latitude','Longtitude','ISO']]
citys['ISO'] = citys['ISO'].astype('str')

In [5]:
data_ready = data_ready.reset_index().reset_index()

In [6]:
country_info=pd.read_table(path+'country_info.txt')

In [7]:
country_info['ISO'] = country_info['ISO'].astype('str')

In [8]:
data_ready = pd.merge(data_ready, country_info[['Country', 'ISO']], left_on='Order Country', right_on='Country', how = 'left').drop(
            columns=['Country'])
data_tmp = pd.merge(data_ready, citys, left_on=['Order City'], right_on=['City'], how = 'left').drop(
            columns=['City'])

data_tmp['ISO_x'] = data_tmp['ISO_x'].astype('str')
data_tmp['ISO_combined'] = np.where(data_tmp['ISO_x']=='nan', data_tmp['ISO_y'], data_tmp['ISO_x'])
data_tmp = data_tmp[data_tmp['ISO_y'] == data_tmp['ISO_combined']]
data_tmp = data_tmp[~data_tmp['index'].duplicated()]
data_ready['Latitude'] = np.nan
data_ready['Longtitude'] = np.nan
data_ready = data_ready.drop(columns = ['ISO'])
data_ready.iloc[data_tmp['index'], -2:] = data_tmp[['Latitude', 'Longtitude']]

latitude_dict = dict(zip(data_ready['Order Region'].unique(), [geocoder.osm(i).latlng[0] for i in data_ready['Order Region'].unique()]))
longitude_dict = dict(zip(data_ready['Order Region'].unique(), [geocoder.osm(i).latlng[1] for i in data_ready['Order Region'].unique()]))
data_ready.loc[data_ready['Latitude'].isnull(), 'Latitude'] = data_ready[data_ready['Latitude'].isnull()]['Order Region'].map(latitude_dict)
data_ready.loc[data_ready['Longtitude'].isnull(), 'Longtitude'] = data_ready[data_ready['Longtitude'].isnull()]['Order Region'].map(longitude_dict)

In [9]:
from geopy import distance
a = data_ready[['Latitude', 'Longtitude']].apply(tuple, axis=1)
b = data_ready[['Depart_Latitude', 'Depart_Longitude']].apply(tuple, axis=1)

In [10]:
distance_array = []
for i in range(a.shape[0]):
    distance_array.append(distance.distance(a[i], b[i]).miles)

In [11]:
data_ready['Distance'] = distance_array

In [12]:
pd.set_option('display.max_columns', 70)

## Order_City_Count

In [13]:
data_ready = data_ready.set_index('Date')
data_ready = data_ready.reset_index()

In [14]:
tmp = data_ready.loc[data_ready[['Date','Order Customer Id']].drop_duplicates().index, ['Date','Order Customer Id', 'Order City']].set_index(['Date', 'Order Customer Id'])
order_city = tmp.unstack()
mask = ~order_city.isna()

In [15]:
def count_city(series):
    a = series.unique()
    return series.map(dict(zip(a, np.arange(a.size))))

In [16]:
order_city = order_city.apply(count_city)
order_city = order_city[mask]
order_city = order_city.stack().reset_index()

In [17]:
order_city = order_city.rename(columns = {order_city.columns[-1]: 'Order_City_Count'})

In [18]:
data_ready = pd.merge(data_ready, order_city, on=['Date', 'Order Customer Id'] , how = 'left')
data_ready['Order_City_Count'] = data_ready['Order_City_Count'] -1
data_ready['Order_City_Count'] = np.where(data_ready['Order_City_Count']<0, 0, data_ready['Order_City_Count'])

## FraudPrevNum

In [19]:
tmp = data_ready.loc[data_ready[['Date','Order Customer Id']].drop_duplicates().index, ['Date','Order Customer Id', 'Order Status']].set_index(['Date', 'Order Customer Id'])
order_status = tmp.unstack()
mask = ~order_status.isna()

In [20]:
order_status[~(order_status == 'SUSPECTED_FRAUD')] = np.nan
order_status[order_status == 'SUSPECTED_FRAUD'] = 1

In [21]:
order_status = order_status.cumsum()
order_status = order_status.shift(1).fillna(method = 'ffill')
order_status = order_status[mask].stack().reset_index()
order_status = order_status.set_index(['Date', 'Order Customer Id'])

In [22]:
tmp['Order Status'] = np.nan
tmp.loc[order_status.index] = order_status
tmp = tmp.fillna(0)

In [23]:
tmp = tmp.rename(columns = {'Order Status': 'FraudPrevNum'})

In [24]:
data_ready = pd.merge(data_ready, tmp, on=['Date', 'Order Customer Id'] , how = 'left')

## AbnormalFreq and OrderCount

In [25]:
tmp = data_ready.loc[data_ready[['Date','Order Customer Id']].drop_duplicates().index, ['Date','Order Customer Id', 'Order Status']].set_index(['Date', 'Order Customer Id'])
order_status = tmp.unstack()
mask = ~order_status.isna()

In [26]:
count_order = order_status.copy()
count_order[~count_order.isna()] = 1
count_order = count_order.cumsum()

In [63]:
# order_status[~((order_status == 'CLOSED') | (order_status == 'PENDING_PAYMENT') 
#                | (order_status == 'PENDING') | (order_status == 'SUSPECTED_FRAUD') | (order_status == 'CANCELED'))] = np.nan
# order_status[(order_status == 'CLOSED') | (order_status == 'PENDING_PAYMENT') | (order_status == 'PENDING') | (order_status == 'SUSPECTED_FRAUD') | (order_status == 'CANCELED')] = 1
order_status[~((order_status == 'SUSPECTED_FRAUD') | (order_status == 'CANCELED') | (order_status == 'CLOSED'))] = np.nan
order_status[(order_status == 'SUSPECTED_FRAUD') | (order_status == 'CANCELED') | (order_status == 'CLOSED')] = 1

In [36]:
AbnormalFreq = order_status/count_order
AbnormalFreq = AbnormalFreq.shift(1).fillna(method = 'ffill')

In [37]:
AbnormalFreq = AbnormalFreq[mask].stack().reset_index()
AbnormalFreq = AbnormalFreq.set_index(['Date', 'Order Customer Id'])

In [38]:
tmp_count = tmp.copy()

In [39]:
tmp['Order Status'] = np.nan
tmp.loc[AbnormalFreq.index] = AbnormalFreq
tmp = tmp.fillna(0)

In [40]:
tmp = tmp.rename(columns = {'Order Status': 'AbnormalFreq'})
data_ready = pd.merge(data_ready, tmp, on=['Date', 'Order Customer Id'] , how = 'left')

In [41]:
count_order = count_order.shift(1).fillna(method = 'ffill')
count_order = count_order[mask].stack().reset_index()
count_order = count_order.set_index(['Date', 'Order Customer Id'])

In [42]:
tmp_count['Order Status'] = np.nan
tmp_count.loc[count_order.index] = count_order
tmp_count = tmp_count.fillna(0)

In [43]:
tmp_count = tmp_count.rename(columns = {'Order Status': 'OrderCount'})
data_ready = pd.merge(data_ready, tmp_count, on=['Date', 'Order Customer Id'] , how = 'left')

## DelayDays

In [44]:
data_ready['DelayDays'] = data_ready['Days for shipping (real)'] - data_ready['Days for shipment (scheduled)']

## One-Hot

In [42]:
payment_type = pd.get_dummies(data_ready['Type'])
payment_type
data_ready = data_ready.drop('Type',axis=1)
data_ready = data_ready.join(payment_type)

In [43]:
delivery_status = pd.get_dummies(data_ready['Delivery Status'])
data_ready = data_ready.drop('Delivery Status',axis=1)
data_ready = data_ready.join(delivery_status)


In [44]:
customer_segment = pd.get_dummies(data_ready['Customer Segment'])
data_ready = data_ready.drop('Customer Segment',axis=1)
data_ready = data_ready.join(customer_segment)


In [45]:
market = pd.get_dummies(data_ready['Market'])
data_ready = data_ready.drop('Market',axis=1)
data_ready = data_ready.join(market)

In [46]:
data_ready = data_ready.sort_values(by="Date", ascending=False)
data_ready['Day Week']= -1  
for index, row in data_ready.iterrows():
    data_ready.at[index, 'Day Week'] = row['Date'].to_pydatetime().weekday()+1

In [47]:
day_week = pd.get_dummies(data_ready['Day Week'])
data_ready = data_ready.drop('Day Week',axis=1)
data_ready = data_ready.join(day_week)

In [48]:
data_ready['date'] = pd.to_datetime(data_ready['Date'], infer_datetime_format=True)
data_ready = data_ready.assign(month=lambda x: x['date'].dt.month)
one_hot = pd.get_dummies(data_ready['month'], prefix='m')
data_ready = data_ready.join(one_hot)
data_ready.drop(columns=['date', 'month'], inplace=True)

In [49]:
data_ready = data_ready.sort_values(by="index", ascending=True)

In [47]:
data_ready['Week_Year'] = data_ready['Date'].dt.to_period('W')

In [48]:
data_ready['Month_Year'] = data_ready['Date'].dt.to_period('M')